In [3]:
import pandas as pd
from dotenv import load_dotenv
from generate_token import generate_token
import os

load_dotenv()
os.environ['YC_IAM_TOKEN'] = generate_token()

In [4]:
from langchain_community.llms import YandexGPT
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

rc_model_uri= "gpt://b1gr0nm9o4sp7b51etoh/yandexgpt-lite/rc"
lite_model_uri= "gpt://b1gr0nm9o4sp7b51etoh/yandexgpt-lite/rc"
pro_model_uri= "gpt://b1gr0nm9o4sp7b51etoh/yandexgpt/latest"

llm = YandexGPT(
    model_uri=pro_model_uri, 
    # sleep_interval=0.1, 
    temperature=0
                )

emb_model_uri = "emb://b1gr0nm9o4sp7b51etoh/text-search-doc/latest"
embeddings = YandexGPTEmbeddings(model_uri=emb_model_uri)

In [5]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate

In [7]:
# Определение других схем ответа
date_range_schema = ResponseSchema(name="date_range",
                                   description="Диапазон дат для интересующих данных")

key_metrics_schema = ResponseSchema(name="key_metrics",
                                    description="Ключевые метрики")

summary_schema = ResponseSchema(name="summary",
                                description="Краткий саммари, подводящий итог по полученным данным. Ответ должен быть коротким абзацем.")

# Формирование списка схем ответа
response_schemas = [
                    date_range_schema,
                    key_metrics_schema,
                    summary_schema]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [12]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"date_range": string  // Диапазон дат для интересующих данных
	"key_metrics": string  // Ключевые метрики
	"summary": string  // Краткий саммари, подводящий итог по полученным данным. Ответ должен быть коротким абзацем.
}
```


In [9]:
table ={
  'Прим:': {
    0: 'Выручка',
    1: 'Административныерасходы',
    2: 'Убытокотоперационнойдеятельности',
    3: 'Процентныефинансовыедоходы',
    4: 'Процентныефинансовыерасходы',
    5: 'Курсовыеразницыипрочиефинансовыедоходыирасходы',
    6: 'Прибыльубытокдоналогообложения',
    7: 'Расход/доходпоналогунаприбыль',
    8: 'Чистаяприбыль/убыток',
    9: 'Прочаясовокупнаяприбыль/убыток',
    10: 'ИТОГОСОВОКУПНАЯПРИБЫЛЬУБЫТОК'
  },
  '31 декабря 2022 года': {
    0: '1 096 667',
    1: '(12 943 994)',
    2: '(11 847 327)',
    3: '10',
    4: '(1185 392 246)',
    5: '(2 091 657)',
    6: '32 397 620',
    7: '(6 505 766)',
    8: '25 891 854',
    9: None,
    10: '25 891 854'
  },
  '31 декабря 2021 года': {
    0: None,
    1: '(13 627 121)',
    2: '(13 627 121)',
    3: '1231 728 850',
    4: '(388 349 279)',
    5: None,
    6: '(66 725 202)',
    7: '1207 903',
    8: '(65 517 299)',
    9: None,
    10: '(65 517 299)'
  },
  'Unnamed: 3': {
    0: None,
    1: None,
    2: None,
    3: '335 251 198',
    4: None,
    5: None,
    6: None,
    7: None,
    8: None,
    9: None,
    10: None
  }
}

In [10]:
from langchain import LLMChain, PromptTemplate
import json

def transform_table(table):
    table = {str(key): value for key, value in table.items()}
    prompt_template = """
    Преобразуй таблицу, выполняя следующие задачи верни json-объект с результатом:
    1. Исправь все опечатки в названиях строк и столбцов.
    2. Если в неименованной колонке ('Unnamed: #') есть число, смести значение вправо по соответствующим колонкам, удаляя крайнее левое значение, и затем удали неименованную колонку.
    3. Удалите скобки из числовых данных и переведите их в числовой тип, например, из (65 517 299) в 65517299. Не должно быть отрицательных значений.
    4. Проверь чтобы ключи могут быть str, int, float, bool, None. Если Tuple то исправь на str.

    Таблица: {table}

    Исправленная таблица:
    """
    prompt = PromptTemplate(input_variables=["table"], template=prompt_template)
    llm_chain = LLMChain(prompt=prompt, llm=llm)

    result = llm_chain.run(table=table)
    return json.loads(result)
    # return result


res = transform_table(table)
print(res)

/home/ilya/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/ilya/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 5 (char 6)

In [13]:

# Определение других схем ответа
date_range_schema = ResponseSchema(name="date_range",
                                   description="Диапазон дат для интересующих данных")

key_metrics_schema = ResponseSchema(name="key_metrics",
                                    description="Ключевые метрики")

summary_schema = ResponseSchema(name="summary",
                                description="Краткий саммари, подводящий итог по полученным данным. Ответ должен быть коротким абзацем.")

# Формирование списка схем ответа
response_schemas = [
                    date_range_schema,
                    key_metrics_schema,
                    summary_schema]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()



template = """
Из следующей таблицы извлеки информацию:
Включи ключевую информацию - диапазон дат, ключевые метрики, краткое описание таблицы.
---
table: 
{table}
---

{format_instructions}

"""

prompt = ChatPromptTemplate.from_template(template=template)


chain = (
    {"table": itemgetter("table"), "format_instructions": itemgetter("format_instructions")}
    | prompt
    | llm
    | output_parser
)

meta_info = chain.invoke({"table": table, "format_instructions": format_instructions})

def combine_table_and_meta(table, meta):
    return {
        "table": table,
        "metadata": meta
    }

final_result = combine_table_and_meta(table, meta_info)
print(json.dumps(final_result, ensure_ascii=False, indent=4))

{
    "table": {
        "Наименование": {
            "0": "Стол}ст,,,,,",
            "1": "отоимошь",
            "2": "Накопленная",
            "3": "Балансовая",
            "4": "2алaпсan",
            "5": "Поступление",
            "6": "Затраты",
            "7": "выоытие",
            "8": "Амортизационные",
            "9": "Прочее,5",
            "10": "Балансовая",
            "11": ",,,,,",
            "12": "2021",
            "13": "Накопленная",
            "14": "2021",
            "15": "ZUzl,(z",
            "16": "Балансовая",
            "17": "Балансовая",
            "18": "zUze",
            "19": "1оступпе,En",
            "20": "Поступление",
            "21": "pa",
            "22": "Амортизационные",
            "23": "выбытии,68",
            "24": "Прочее,11",
            "25": "Балансовая",
            "26": "Стоимость",
            "27": "2022",
            "28": "2022",
            "29": "Балансовая"
        },
        "показателя,Активы": {
         

In [62]:
print(result)

{'date_range': '31 декабря 2022 года, 31 декабря 2021 года', 'key_metrics': 'Выручка, Административные расходы, Убыток от операционной деятельности, Процентные финансовые доходы, Процентные финансовые расходы, Курсовые разницы и прочие финансовые доходы и расходы, Прибыль (убыток) до налогообложения, Расход/доход по налогу на прибыль, Чистая прибыль/убыток, Прочая совокупная прибыль/убыток', 'summary': 'В таблице представлены финансовые показатели за два года. За 2022 год компания получила чистую прибыль в размере 25 891 854, что является улучшением по сравнению с убытком в размере 65 517 299 за 2021 год.'}


In [14]:
def combine_table_and_meta(table, meta):
    return {
        "table": table,
        "metadata": meta
    }

final_result = combine_table_and_meta(table, meta_info)
print(json.dumps(final_result, ensure_ascii=False, indent=4))

{
    "table": {
        "Наименование": {
            "0": "Стол}ст,,,,,",
            "1": "отоимошь",
            "2": "Накопленная",
            "3": "Балансовая",
            "4": "2алaпсan",
            "5": "Поступление",
            "6": "Затраты",
            "7": "выоытие",
            "8": "Амортизационные",
            "9": "Прочее,5",
            "10": "Балансовая",
            "11": ",,,,,",
            "12": "2021",
            "13": "Накопленная",
            "14": "2021",
            "15": "ZUzl,(z",
            "16": "Балансовая",
            "17": "Балансовая",
            "18": "zUze",
            "19": "1оступпе,En",
            "20": "Поступление",
            "21": "pa",
            "22": "Амортизационные",
            "23": "выбытии,68",
            "24": "Прочее,11",
            "25": "Балансовая",
            "26": "Стоимость",
            "27": "2022",
            "28": "2022",
            "29": "Балансовая"
        },
        "показателя,Активы": {
         

In [15]:
import glob
import json

def process_table(table):
    chain = (
        {"table": itemgetter("table"), "format_instructions": itemgetter("format_instructions")}
        | prompt
        | llm
        | output_parser
    )

    meta_info = chain.invoke({"table": table, "format_instructions": format_instructions})

    def combine_table_and_meta(table, meta):
        return {
            "table": table,
            "metadata": meta
        }

    final_result = combine_table_and_meta(table, meta_info)
    return final_result

optput_data_dir = './../data/output/'

dirs = [name for name in os.listdir('./../data/output/') if os.path.isdir(os.path.join('./../data/output/', name))]

# Process each directory
for dir in dirs:
    path = os.path.join('./../data/output/', dir)
    company_name = dir.split('_')[0]
    csv_files = glob.glob(f'{path}/*.csv')

    for csv_file in csv_files:
        try:
            table_name = csv_file.split('/')[-1].split('.')[0]
            final_path = f'{path}/distilled/{company_name}_{table_name}.json'
            if os.path.exists(final_path):
                print(f'Skipped {csv_file} because {final_path} already exists')
                continue
            
            df = pd.read_csv(csv_file, delim_whitespace=True, on_bad_lines='skip')
            table = df.to_dict()
            table = transform_table(table)
            results = process_table(table)
            results['company_name'] = company_name
            
            os.makedirs(f'{path}/distilled',  exist_ok=True)
            
            with open(final_path, 'w') as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
                
            print(f'Processed {csv_file} and saved to {final_path}')
        except Exception as e:
            print(f'Error processing {csv_file}: {e}')
    

Skipped ./../data/output/Мейл.Ру Финанс_Отчетность и аудиторское заключение за 2022 год Мэйл.Ру Финанс/table_page23.csv because ./../data/output/Мейл.Ру Финанс_Отчетность и аудиторское заключение за 2022 год Мэйл.Ру Финанс/distilled/Мейл.Ру Финанс_table_page23.json already exists
Error processing ./../data/output/Мейл.Ру Финанс_Отчетность и аудиторское заключение за 2022 год Мэйл.Ру Финанс/table_page3.csv: No columns to parse from file
Error processing ./../data/output/Мейл.Ру Финанс_Отчетность и аудиторское заключение за 2022 год Мэйл.Ру Финанс/table_page12.csv: Expecting property name enclosed in double quotes: line 3 column 9 (char 28)
Skipped ./../data/output/Мейл.Ру Финанс_Отчетность и аудиторское заключение за 2022 год Мэйл.Ру Финанс/table_page15.csv because ./../data/output/Мейл.Ру Финанс_Отчетность и аудиторское заключение за 2022 год Мэйл.Ру Финанс/distilled/Мейл.Ру Финанс_table_page15.json already exists
Error processing ./../data/output/Мейл.Ру Финанс_Отчетность и аудиторско